Predict whether the loan should be agreed using a decision tree algorithm

In [152]:
#import packet
from pandas import Series,DataFrame
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from datetime import datetime
import time

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

#1.获取数据
#2.基本数据处理
#2.1 缺失值处理
#2.2 确定特征值,目标值
#2.3 分割数据
#3.特征工程(标准化)
#4.机器学习(逻辑回归)
#5.模型评估

In [114]:
# 1.获取数据

data = pd.read_csv("SBAnational.csv")
data.head()

D:\DT228-4Soft\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,...,N,Y,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00"
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,...,N,Y,NaN,31-May-97,"$40,000.00",$0.00,P I F,$0.00,"$40,000.00","$32,000.00"
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,28-Feb-97,1997,...,N,N,NaN,31-Dec-97,"$287,000.00",$0.00,P I F,$0.00,"$287,000.00","$215,250.00"
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,28-Feb-97,1997,...,N,Y,NaN,30-Jun-97,"$35,000.00",$0.00,P I F,$0.00,"$35,000.00","$28,000.00"
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,28-Feb-97,1997,...,N,N,NaN,14-May-97,"$229,000.00",$0.00,P I F,$0.00,"$229,000.00","$229,000.00"


In [115]:
column_name = ["State","NAICS","ApprovalDate", "Term", "NoEmp", "NewExist", "CreateJob", "RetainedJob", "FranchiseCode", "UrbanRural", "LowDoc", "DisbursementDate", "DisbursementGross", "GrAppv", "SBA_Appv", "MIS_Status"]
data = data[column_name]
data.head(10)
data.dtypes
data.isnull().sum()

State                  14
NAICS                   0
ApprovalDate            0
Term                    0
NoEmp                   0
NewExist              136
CreateJob               0
RetainedJob             0
FranchiseCode           0
UrbanRural              0
LowDoc               2582
DisbursementDate     2368
DisbursementGross       0
GrAppv                  0
SBA_Appv                0
MIS_Status           1997
dtype: int64

In [116]:
data = data.replace(to_replace="?", value=np.NaN)
data = data.dropna()

In [117]:
# 2.基本数据处理
# 2.1 缺失值处理
data.isnull().sum()
data = data.replace(to_replace="?", value=np.NaN)
data = data.dropna()


#data clearn: Converts string data to numeric types

cleanup_boolean_value = {
                "MIS_Status":{"P I F":1,"CHGOFF":0}}

data = data.replace(cleanup_boolean_value)
data.head(10)
data.dtypes

State                 object
NAICS                  int64
ApprovalDate          object
Term                   int64
NoEmp                  int64
NewExist             float64
CreateJob              int64
RetainedJob            int64
FranchiseCode          int64
UrbanRural             int64
LowDoc                object
DisbursementDate      object
DisbursementGross     object
GrAppv                object
SBA_Appv              object
MIS_Status             int64
dtype: object

In [118]:
data.head(10)

,State,NAICS,ApprovalDate,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,LowDoc,DisbursementDate,DisbursementGross,GrAppv,SBA_Appv,MIS_Status
0,IN,451120,28-Feb-97,84,4,2.0,0,0,1,0,Y,28-Feb-99,"$60,000.00","$60,000.00","$48,000.00",1
1,IN,722410,28-Feb-97,60,2,2.0,0,0,1,0,Y,31-May-97,"$40,000.00","$40,000.00","$32,000.00",1
2,IN,621210,28-Feb-97,180,7,1.0,0,0,1,0,N,31-Dec-97,"$287,000.00","$287,000.00","$215,250.00",1
3,OK,0,28-Feb-97,60,2,1.0,0,0,1,0,Y,30-Jun-97,"$35,000.00","$35,000.00","$28,000.00",1
4,FL,0,28-Feb-97,240,14,1.0,7,7,1,0,N,14-May-97,"$229,000.00","$229,000.00","$229,000.00",1
5,CT,332721,28-Feb-97,120,19,1.0,0,0,1,0,N,30-Jun-97,"$517,000.00","$517,000.00","$387,750.00",1
6,NJ,0,2-Jun-80,45,45,2.0,0,0,0,0,N,22-Jul-80,"$600,000.00","$600,000.00","$499,998.00",0
7,FL,811118,28-Feb-97,84,1,2.0,0,0,1,0,Y,30-Jun-98,"$45,000.00","$45,000.00","$36,000.00",1
8,FL,721310,28-Feb-97,297,2,2.0,0,0,1,0,N,31-Jul-97,"$305,000.00","$305,000.00","$228,750.00",1
9,CT,0,28-Feb-97,84,3,2.0,0,0,1,0,Y,30-Apr-97,"$70,000.00","$70,000.00","$56,000.00",1


In [119]:
cleanup_boolean_value = {"LowDoc":{"Y":1, "N":0}}
data = data.replace(cleanup_boolean_value)

data.dtypes

State                 object
NAICS                  int64
ApprovalDate          object
Term                   int64
NoEmp                  int64
NewExist             float64
CreateJob              int64
RetainedJob            int64
FranchiseCode          int64
UrbanRural             int64
LowDoc                object
DisbursementDate      object
DisbursementGross     object
GrAppv                object
SBA_Appv              object
MIS_Status             int64
dtype: object

In [120]:
len(data["LowDoc"])


892302

In [125]:
data.groupby(['LowDoc'],as_index=False)['LowDoc'].agg({'cnt':'count'})

,LowDoc,cnt
0,0,778729
1,1,110162
2,0,1489
3,1,1
4,A,492
5,C,757
6,R,74
7,S,598


In [5]:
#将state转化为标签类型，然后生成一个数字类型的state code
data["State"] = data["State"].astype('category')
data.dtypes
data["State_code"] = data["State"].cat.codes
data.head(10)

,State,NAICS,ApprovalDate,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,LowDoc,DisbursementDate,DisbursementGross,GrAppv,SBA_Appv,MIS_Status,State_code
0,IN,451120,28-Feb-97,84,4,2.0,0,0,1,0,1,28-Feb-99,"$60,000.00","$60,000.00","$48,000.00",1,15
1,IN,722410,28-Feb-97,60,2,2.0,0,0,1,0,1,31-May-97,"$40,000.00","$40,000.00","$32,000.00",1,15
2,IN,621210,28-Feb-97,180,7,1.0,0,0,1,0,0,31-Dec-97,"$287,000.00","$287,000.00","$215,250.00",1,15
3,OK,0,28-Feb-97,60,2,1.0,0,0,1,0,1,30-Jun-97,"$35,000.00","$35,000.00","$28,000.00",1,36
4,FL,0,28-Feb-97,240,14,1.0,7,7,1,0,0,14-May-97,"$229,000.00","$229,000.00","$229,000.00",1,9
5,CT,332721,28-Feb-97,120,19,1.0,0,0,1,0,0,30-Jun-97,"$517,000.00","$517,000.00","$387,750.00",1,6
6,NJ,0,2-Jun-80,45,45,2.0,0,0,0,0,0,22-Jul-80,"$600,000.00","$600,000.00","$499,998.00",0,31
7,FL,811118,28-Feb-97,84,1,2.0,0,0,1,0,1,30-Jun-98,"$45,000.00","$45,000.00","$36,000.00",1,9
8,FL,721310,28-Feb-97,297,2,2.0,0,0,1,0,0,31-Jul-97,"$305,000.00","$305,000.00","$228,750.00",1,9
9,CT,0,28-Feb-97,84,3,2.0,0,0,1,0,1,30-Apr-97,"$70,000.00","$70,000.00","$56,000.00",1,6


In [6]:
data_removeDo = data
data_removeDo.head()

,State,NAICS,ApprovalDate,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,LowDoc,DisbursementDate,DisbursementGross,GrAppv,SBA_Appv,MIS_Status,State_code
0,IN,451120,28-Feb-97,84,4,2.0,0,0,1,0,1,28-Feb-99,"$60,000.00","$60,000.00","$48,000.00",1,15
1,IN,722410,28-Feb-97,60,2,2.0,0,0,1,0,1,31-May-97,"$40,000.00","$40,000.00","$32,000.00",1,15
2,IN,621210,28-Feb-97,180,7,1.0,0,0,1,0,0,31-Dec-97,"$287,000.00","$287,000.00","$215,250.00",1,15
3,OK,0,28-Feb-97,60,2,1.0,0,0,1,0,1,30-Jun-97,"$35,000.00","$35,000.00","$28,000.00",1,36
4,FL,0,28-Feb-97,240,14,1.0,7,7,1,0,0,14-May-97,"$229,000.00","$229,000.00","$229,000.00",1,9


In [ ]:
#remobve $
def removeDollorS(dollorString):
    noDollor = [ ]
    for value in dollorString:
        #get value without $
        noDollor.append(value[1:])
        #Convert the list to a Series type of one-dimensional data
        dollorSer = pd.Series(noDollor)
        
    return dollorSer


'''
data_removeDo["DisbursementGross"] = removeDollorS(data_removeDo["DisbursementGross"]) 
这种方法运行速度太慢了

'''

In [8]:
#remobve $
data_removeDo['DisbursementGross'] = data_removeDo['DisbursementGross'].map(lambda x: str(x)[1:])
data_removeDo['GrAppv'] = data_removeDo['GrAppv'].map(lambda x: str(x)[1:])
data_removeDo['SBA_Appv'] = data_removeDo['SBA_Appv'].map(lambda x: str(x)[1:])

data_removeDo.head(10)

,State,NAICS,ApprovalDate,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,LowDoc,DisbursementDate,DisbursementGross,GrAppv,SBA_Appv,MIS_Status,State_code
0,IN,451120,28-Feb-97,84,4,2.0,0,0,1,0,1,28-Feb-99,"60,000.00","60,000.00","48,000.00",1,15
1,IN,722410,28-Feb-97,60,2,2.0,0,0,1,0,1,31-May-97,"40,000.00","40,000.00","32,000.00",1,15
2,IN,621210,28-Feb-97,180,7,1.0,0,0,1,0,0,31-Dec-97,"287,000.00","287,000.00","215,250.00",1,15
3,OK,0,28-Feb-97,60,2,1.0,0,0,1,0,1,30-Jun-97,"35,000.00","35,000.00","28,000.00",1,36
4,FL,0,28-Feb-97,240,14,1.0,7,7,1,0,0,14-May-97,"229,000.00","229,000.00","229,000.00",1,9
5,CT,332721,28-Feb-97,120,19,1.0,0,0,1,0,0,30-Jun-97,"517,000.00","517,000.00","387,750.00",1,6
6,NJ,0,2-Jun-80,45,45,2.0,0,0,0,0,0,22-Jul-80,"600,000.00","600,000.00","499,998.00",0,31
7,FL,811118,28-Feb-97,84,1,2.0,0,0,1,0,1,30-Jun-98,"45,000.00","45,000.00","36,000.00",1,9
8,FL,721310,28-Feb-97,297,2,2.0,0,0,1,0,0,31-Jul-97,"305,000.00","305,000.00","228,750.00",1,9
9,CT,0,28-Feb-97,84,3,2.0,0,0,1,0,1,30-Apr-97,"70,000.00","70,000.00","56,000.00",1,6


In [67]:
#Converts the timestamp to a numeric type
data_time_num = data_removeDo
data_time_num["ApprovalDate"] = pd.to_datetime(data_time_num["ApprovalDate"]).dt.strftime('%m%Y')
data_time_num["DisbursementDate"] = pd.to_datetime(data_time_num["DisbursementDate"]).dt.strftime('%m%Y')
data_time_num

,State,NAICS,ApprovalDate,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,LowDoc,DisbursementDate,DisbursementGross,GrAppv,SBA_Appv,MIS_Status,State_code
0,IN,451120,021997,84,4,2.0,0,0,1,0,1,021999,"60,000.00","60,000.00","48,000.00",1,15
1,IN,722410,021997,60,2,2.0,0,0,1,0,1,051997,"40,000.00","40,000.00","32,000.00",1,15
2,IN,621210,021997,180,7,1.0,0,0,1,0,0,121997,"287,000.00","287,000.00","215,250.00",1,15
3,OK,0,021997,60,2,1.0,0,0,1,0,1,061997,"35,000.00","35,000.00","28,000.00",1,36
4,FL,0,021997,240,14,1.0,7,7,1,0,0,051997,"229,000.00","229,000.00","229,000.00",1,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899159,OH,451120,021997,60,6,1.0,0,0,1,0,0,091997,"70,000.00","70,000.00","56,000.00",1,35
899160,OH,451130,021997,60,6,1.0,0,0,1,0,0,101997,"85,000.00","85,000.00","42,500.00",1,35
899161,CA,332321,021997,108,26,1.0,0,0,1,0,0,091997,"300,000.00","300,000.00","225,000.00",1,4
899162,HI,0,021997,60,6,1.0,0,0,1,0,1,031997,"75,000.00","75,000.00","60,000.00",0,11


In [187]:
final_column_name = ["State_code","NAICS","ApprovalDate", "Term", "NoEmp", "NewExist", "CreateJob", "RetainedJob", "FranchiseCode", "UrbanRural", "LowDoc", "DisbursementDate", "DisbursementGross", "GrAppv", "SBA_Appv", "MIS_Status"]
data_cleaned = data_time_num[final_column_name]

data_cleaned
data_cleaned.dtypes



State_code              int8
NAICS                  int64
ApprovalDate          object
Term                   int64
NoEmp                  int64
NewExist             float64
CreateJob              int64
RetainedJob            int64
FranchiseCode          int64
UrbanRural             int64
LowDoc                object
DisbursementDate      object
DisbursementGross     object
GrAppv                object
SBA_Appv              object
MIS_Status             int64
dtype: object

In [132]:
# 2.2 确定特征值,目标值
x = data_cleaned.iloc[:, 11:12]
x.head()
y = data_cleaned["MIS_Status"]
y.head()
# 2.3 分割数据
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=22)


In [133]:
# 3.特征工程(标准化)
transfer = StandardScaler()
x_train = transfer.fit_transform(x_train)
x_test = transfer.transform(x_test)

In [177]:
data_cleaned
data_cleaned.dtypes
len(data_cleaned)

892302

In [178]:
data_cleaned.dtypes

State_code              int8
NAICS                  int64
ApprovalDate          object
Term                   int64
NoEmp                  int64
NewExist             float64
CreateJob              int64
RetainedJob            int64
FranchiseCode          int64
UrbanRural             int64
LowDoc                object
DisbursementDate      object
DisbursementGross     object
GrAppv                object
SBA_Appv              object
MIS_Status             int64
dtype: object

In [188]:
to_numeric_col = ["ApprovalDate","LowDoc","DisbursementDate"]
for name in to_numeric_col:
    data_cleaned= data_cleaned[pd.to_numeric(data_cleaned[name],errors='coerce').notnull()]
data_cleaned

,State_code,NAICS,ApprovalDate,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,LowDoc,DisbursementDate,DisbursementGross,GrAppv,SBA_Appv,MIS_Status
0,15,451120,021997,84,4,2.0,0,0,1,0,1,021999,"60,000.00","60,000.00","48,000.00",1
1,15,722410,021997,60,2,2.0,0,0,1,0,1,051997,"40,000.00","40,000.00","32,000.00",1
2,15,621210,021997,180,7,1.0,0,0,1,0,0,121997,"287,000.00","287,000.00","215,250.00",1
3,36,0,021997,60,2,1.0,0,0,1,0,1,061997,"35,000.00","35,000.00","28,000.00",1
4,9,0,021997,240,14,1.0,7,7,1,0,0,051997,"229,000.00","229,000.00","229,000.00",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899159,35,451120,021997,60,6,1.0,0,0,1,0,0,091997,"70,000.00","70,000.00","56,000.00",1
899160,35,451130,021997,60,6,1.0,0,0,1,0,0,101997,"85,000.00","85,000.00","42,500.00",1
899161,4,332321,021997,108,26,1.0,0,0,1,0,0,091997,"300,000.00","300,000.00","225,000.00",1
899162,11,0,021997,60,6,1.0,0,0,1,0,1,031997,"75,000.00","75,000.00","60,000.00",0


,State_code,NAICS,ApprovalDate,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,LowDoc,DisbursementDate,DisbursementGross,GrAppv,SBA_Appv,MIS_Status
0,15,451120,021997,84,4,2.0,0,0,1,0,1,021999,"60,000.00","60,000.00","48,000.00",1
1,15,722410,021997,60,2,2.0,0,0,1,0,1,051997,"40,000.00","40,000.00","32,000.00",1
2,15,621210,021997,180,7,1.0,0,0,1,0,0,121997,"287,000.00","287,000.00","215,250.00",1
3,36,0,021997,60,2,1.0,0,0,1,0,1,061997,"35,000.00","35,000.00","28,000.00",1
4,9,0,021997,240,14,1.0,7,7,1,0,0,051997,"229,000.00","229,000.00","229,000.00",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899159,35,451120,021997,60,6,1.0,0,0,1,0,0,091997,"70,000.00","70,000.00","56,000.00",1
899160,35,451130,021997,60,6,1.0,0,0,1,0,0,101997,"85,000.00","85,000.00","42,500.00",1
899161,4,332321,021997,108,26,1.0,0,0,1,0,0,091997,"300,000.00","300,000.00","225,000.00",1
899162,11,0,021997,60,6,1.0,0,0,1,0,1,031997,"75,000.00","75,000.00","60,000.00",0


In [182]:
data_cleaned.dtypes

State_code              int8
NAICS                  int64
ApprovalDate          object
Term                   int64
NoEmp                  int64
NewExist             float64
CreateJob              int64
RetainedJob            int64
FranchiseCode          int64
UrbanRural             int64
LowDoc                object
DisbursementDate      object
DisbursementGross     object
GrAppv                object
SBA_Appv              object
MIS_Status             int64
dtype: object

In [204]:
# 2.2 确定特征值,目标值
x = data_cleaned.iloc[:, 15:15]
x.head()
y = data_cleaned["MIS_Status"]
y.head()
# 2.3 分割数据
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=22)

In [205]:
# 3.特征工程(标准化)
transfer = StandardScaler()
x_train = transfer.fit_transform(x_train)
x_test = transfer.transform(x_test)

ValueError: could not convert string to float: '251,250.00 '

In [210]:
#remove "," 
data_removeDo['SBA_Appv'] = data_removeDo['SBA_Appv'].map(lambda x: x.replace(',',''))


In [215]:
name_list=["DisbursementGross","GrAppv"]
for name in name_list:
    data_cleaned[name] = data_cleaned[name].map(lambda x: x.replace(',',''))

In [216]:
data_cleaned

,State_code,NAICS,ApprovalDate,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,LowDoc,DisbursementDate,DisbursementGross,GrAppv,SBA_Appv,MIS_Status
0,15,451120,021997,84,4,2.0,0,0,1,0,1,021999,60000.00,60000.00,48000.00,1
1,15,722410,021997,60,2,2.0,0,0,1,0,1,051997,40000.00,40000.00,32000.00,1
2,15,621210,021997,180,7,1.0,0,0,1,0,0,121997,287000.00,287000.00,215250.00,1
3,36,0,021997,60,2,1.0,0,0,1,0,1,061997,35000.00,35000.00,28000.00,1
4,9,0,021997,240,14,1.0,7,7,1,0,0,051997,229000.00,229000.00,229000.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899159,35,451120,021997,60,6,1.0,0,0,1,0,0,091997,70000.00,70000.00,56000.00,1
899160,35,451130,021997,60,6,1.0,0,0,1,0,0,101997,85000.00,85000.00,42500.00,1
899161,4,332321,021997,108,26,1.0,0,0,1,0,0,091997,300000.00,300000.00,225000.00,1
899162,11,0,021997,60,6,1.0,0,0,1,0,1,031997,75000.00,75000.00,60000.00,0


In [217]:
# 2.2 Determine eigenvalues and target values
x = data_cleaned.iloc[:, 1:15]
x.head()
y = data_cleaned["MIS_Status"]
y.head()
# 2.3 split dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=22)

In [218]:
# 3.Feature Engineering (Standardization)
transfer = StandardScaler()
x_train = transfer.fit_transform(x_train)
x_test = transfer.transform(x_test)

In [219]:
# 4.Machine Learning (Logical Regression)
estimator = LogisticRegression()
estimator.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [220]:
y_predict = estimator.predict(x_test)
y_predict
estimator.score(x_test, y_test)

0.8354597566892487

In [225]:
#LinearRegression
from sklearn.linear_model import LinearRegression,SGDRegressor
estimator = SGDRegressor(max_iter=1000)
estimator.fit(x_train, y_train)


SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='squared_loss', max_iter=1000,
             n_iter_no_change=5, penalty='l2', power_t=0.25, random_state=None,
             shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [226]:
y_predict = estimator.predict(x_test)
estimator.score(x_test, y_test)

0.12124643711618766